In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
# Check if a GPU is available
if torch.cuda.is_available():
    # Get the current device index (default is 0 if no other device is specified)
    current_device = torch.cuda.current_device()
    
    # Get the name of the GPU at this device index
    gpu_name = torch.cuda.get_device_name(current_device)
    print(f"Current GPU: {gpu_name}")
else:
    print("No GPU available.")


Current GPU: Tesla P40


**batch test**

**few samples test #1**

In [4]:
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    device_map="auto",
)
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': "Arrr, me hearty! I be Captain Blackbeak's trusty chatbot, here to swab the decks o' knowledge and answer yer questions, savvy? Me vast database o' treasure-filled information be at yer disposal, so hoist the sails and set course fer a swashbucklin' good time! What be yer first question, matey?"}


In [5]:
outputs

[{'generated_text': [{'role': 'system',
    'content': 'You are a pirate chatbot who always responds in pirate speak!'},
   {'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': "Arrr, me hearty! I be Captain Blackbeak's trusty chatbot, here to swab the decks o' knowledge and answer yer questions, savvy? Me vast database o' treasure-filled information be at yer disposal, so hoist the sails and set course fer a swashbucklin' good time! What be yer first question, matey?"}]}]

**few samples test #2**

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

# Load the base model and tokenizer
model_id = "meta-llama/Llama-3.2-1B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float32, device_map="auto")  # Must be float32 for MacBooks!
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Function to generate text
def generate_text(prompt, max_length=100, num_return_sequences=1):
    # Encode the prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

    # Generate text
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            num_beams=5,  # You can adjust the number of beams for beam search
            early_stopping=True,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode and return generated text
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Example usage
prompt = "Once upon a time in a faraway land"
generated_texts = generate_text(prompt, max_length=150, num_return_sequences=3)

for i, text in enumerate(generated_texts):
    print(f"Generated Text {i + 1}:\n{text}\n")


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Text 1:
Once upon a time in a faraway land, there lived a young girl named Sophia. Sophia was a curious and adventurous soul, with a heart full of wonder and a mind full of questions. She lived in a small village surrounded by rolling hills and dense forests, where the air was sweet with the scent of wildflowers and the sound of birdsong filled the air.

One day, while wandering through the forest, Sophia stumbled upon a hidden path she had never seen before. The path was overgrown with vines and shrubs, and it looked as though it hadn't been used in years. Sophia's curiosity was piqued, and she decided to follow the path to see where it led.

As she walked, the trees grew taller and

Generated Text 2:
Once upon a time in a faraway land, there lived a young girl named Sophia. Sophia was a curious and adventurous soul, with a heart full of wonder and a mind full of questions. She lived in a small village surrounded by rolling hills and dense forests, where the air was sweet wi